In [1]:
import mitsuba as mi
import drjit as dr
from os import chdir, listdir
from PIL import Image
import matplotlib.gridspec as gridspec
from mpl_toolkits.axes_grid1 import ImageGrid
import matplotlib.pyplot as plt
import os
import numpy as np
import torch as ch
mi.set_variant("cuda_ad_rgb")
from mitsuba import ScalarTransform4f as T

In [2]:
# # yellow=np.array([0.888/255.0, 0.468/255.0, 0.0/255.0])
# height = 1024
# width = height
# channels = 3
# image_blank = np.zeros((height, width, channels), np.uint8)
# image_blank[::] = [0.888*255.0, 0.468*255.0, 0.0*255.0]
# # image_filename = "image_color.jpeg"
# # image_blank.save(image_filename)
# mi.util.convert_to_bitmap(image_blank)
# mi.util.write_bitmap("yellow.png", image_blank)


In [4]:
scene = mi.load_file("../scenes/cube_scene_r1/cube_scene_r1.xml")
img = mi.render(scene, spp=128)
mi.util.convert_to_bitmap(img)


Bitmap[
  pixel_format = rgb,
  component_format = uint8,
  size = [512, 512],
  srgb_gamma = 1,
  struct = Struct<3>[
    uint8 R; // @0, normalized, gamma, premultiplied alpha
    uint8 G; // @1, normalized, gamma, premultiplied alpha
    uint8 B; // @2, normalized, gamma, premultiplied alpha
  ],
  data = [ 768 KiB of image data ]
]

In [5]:
params = mi.traverse(scene)
print(params)

SceneParameters[
  ------------------------------------------------------------------------------------------------------------
  Name                                                     Flags    Type  Parent
  ------------------------------------------------------------------------------------------------------------
  PerspectiveCamera.near_clip                                       float PerspectiveCamera
  PerspectiveCamera.far_clip                                        float PerspectiveCamera
  PerspectiveCamera.shutter_open                                    float PerspectiveCamera
  PerspectiveCamera.shutter_open_time                               float PerspectiveCamera
  PerspectiveCamera.film.size                                       ScalarVector2u HDRFilm
  PerspectiveCamera.film.crop_size                                  ScalarVector2u HDRFilm
  PerspectiveCamera.film.crop_offset                                ScalarPoint2u HDRFilm
  PerspectiveCamera.x_fov               

In [7]:
bmp = mi.Bitmap('../scenes/cube_scene_r1/textures/person_tex/16_cam_seq_noise/tex_0.png')
fn = '../scenes/cube_scene_r1/textures/person_tex/16_cam_seq_noise/tex_0.png'

In [11]:
heightmap_texture = mi.load_dict({
    'type': 'bitmap',
    'id': 'heightmap_texture',
    'filename': fn,
    'raw': True
})
    # 'bitmap': mi.Bitmap(dr.zeros(mi.TensorXf, initial_heightmap_resolution)),

In [ ]:
ht = mi.traverse(heightmap_texture)
print(ht)
params["mat-Material.brdf_0.base_color.data" ] = ht['data']
params.update()
img = mi.render(scene, params=params, spp=128)
mi.util.convert_to_bitmap(img)

In [5]:
icosphere = mi.load_dict({
    'type': 'scene',
    # The keys below correspond to object IDs and can be chosen arbitrarily
    'icosphere': {
        'type': 'ply',
        'filename': "../scenes/cube_scene/meshes/sphere_mid.ply"
    },
})

In [6]:
ip = mi.traverse(icosphere)
print(ip)

SceneParameters[
  ---------------------------------------------------------------------------------------
  Name                                Flags    Type  Parent
  ---------------------------------------------------------------------------------------
  icosphere.bsdf.reflectance.value    ∂        Float UniformSpectrum
  icosphere.vertex_count                       int   PLYMesh
  icosphere.face_count                         int   PLYMesh
  icosphere.faces                              UInt  PLYMesh
  icosphere.vertex_positions          ∂, D     Float PLYMesh
  icosphere.vertex_normals            ∂, D     Float PLYMesh
  icosphere.vertex_texcoords          ∂        Float PLYMesh
]


In [7]:
# we have a half-dome shaped mesh sitting over the focus object / cube
# take vertices of this half-dome and place a camera @ a desired vertex that focuses on cube
# render from each(or randomly choose) a placed camera to obtain various viewing angles
ipv = np.array(ip["icosphere.vertex_positions"])
ipv  = np.reshape(ipv,(int(len(ipv)/3),3))

In [8]:
# get strata / levels of cam heights
strata = np.array(list(set(np.round(ipv[:,1],3))))
# all cam positions in strata[3]
cam_pos_ring = ipv[np.where(np.round(ipv,3)[:,1] == strata[2])]
print(cam_pos_ring)

[[  0.022794     4.201864   -10.128655  ]
 [  3.9048107    4.201864    -9.356474  ]
 [  7.195825     4.201864    -7.157488  ]
 [  9.39481      4.201864    -3.8664734 ]
 [ 10.166989     4.201864     0.01554291]
 [  9.394807     4.201864     3.8975585 ]
 [  7.195821     4.201864     7.188572  ]
 [  3.904807     4.201864     9.387556  ]
 [  0.02279151   4.201864    10.159736  ]
 [ -3.8592234    4.201864     9.387553  ]
 [ -7.150236     4.201864     7.1885676 ]
 [ -9.34922      4.201864     3.8975532 ]
 [-10.1214       4.201864     0.0155379 ]
 [-10.1214       4.201864     0.0155379 ]
 [ -9.349217     4.201864    -3.866477  ]
 [ -7.150231     4.201864    -7.157489  ]
 [ -3.859216     4.201864    -9.356472  ]]


In [9]:

def load_sensor(r, phi, theta, xpos):
    # Apply two rotations to convert from spherical coordinates to world 3D coordinates.
    origin = T.rotate([0, 0,  1], phi).rotate([0, 1, 0], theta) @ mi.ScalarPoint3f([0, 0, r])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.3077,
        'to_world': T.look_at(
            origin=origin,
            target=[0, -0.5, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 16
        },
        'film': {
            'type': 'hdrfilm',
            'width': 256,
            'height': 256,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgb',
        },
    })

def load_sensor_at_position(x,y,z):  
    origin = mi.ScalarPoint3f([x,y,z])

    return mi.load_dict({
        'type': 'perspective',
        'fov': 39.3077,
        'to_world': T.look_at(
            origin=origin,
            target=[0, -0.5, 0],
            up=[0, 1, 0]
        ),
        'sampler': {
            'type': 'independent',
            'sample_count': 16
        },
        'film': {
            'type': 'hdrfilm',
            'width': 512,
            'height': 512,
            'rfilter': {
                'type': 'tent',
            },
            'pixel_format': 'rgb',
        },
    })    

from dt2 import generate_cube_scene_orbit_cam_positions
from dt2 import save_adv_image_preds
camera_positions = generate_cube_scene_orbit_cam_positions()
cam_key = 'PerspectiveCamera.to_world'
for i in range(0, len(camera_positions)):
    params[cam_key].matrix = camera_positions[i].matrix
    params.update()
    img =  mi.render(scene, params=params, spp=256, sensor=0, seed=i+1)
    rendered_img_path = f"/nvmescratch/mhull32/robust-models-transfer/renders/render_{i}.png"
    mi.util.write_bitmap(rendered_img_path, data=img)

In [ ]:
p = mi.traverse(scene)
sensor_count = len(cam_pos_ring)
# radius = 10.0
# phis = [10.0 * i for i in range(sensor_count)]
# theta = 60.0
# y = 5
# ys = [1,3,5] #low, mid-level, overhead
ys = [2]
sensors = np.array([])
for y in ys:
    _sensors = np.array([load_sensor_at_position(p[0], p[1], p[2]) for p in cam_pos_ring])
    sensors = np.append(sensors, _sensors)
sensors = sensors.flatten().tolist()
paths = [os.path.join("/nvmescratch/mhull32/robust-models-transfer/renders", f"{i}.png") for i in range(0,len(sensors))]
images = [mi.render(scene, p, spp=8, sensor=sensor) for sensor in sensors]
_ = [mi.util.write_bitmap(paths[i],images[i]) for i in range(0, len(paths))] 

In [ ]:
sensors[0].world_transform()

In [ ]:
{
    "tags": [
        "hide_input",
    ]
}
import glob
im_path = '/nvmescratch/mhull32/robust-models-transfer/renders/'
chdir(im_path)
files = glob.glob(f"{os.getcwd()}/*.png")
chdir("../")
# im = paths
im = files
images = [Image.open(f) for f in im]
num_rows = 5
num_cols = 4
fig = plt.figure(dpi=256)
gs = gridspec.GridSpec(num_rows, num_cols)

ax = [plt.subplot(gs[i]) for i in range(len(images))]

for i,im in enumerate(images):
    ax[i].imshow(im)
    ax[i].axis('off')
plt.subplots_adjust(wspace=0.01,hspace=-0.71)
plt.show()

Get Camera position

In [ ]:
p = mi.traverse(scene)
cam_position = p['PerspectiveCamera_10.to_world']
print(cam_position)

In [ ]:
p = mi.traverse(scene)
print(p)

Translate Camera closer to the stop sign, update parameters, re-render

In [ ]:
%%capture
m = cam_position.matrix
m[3][2][0] = -2.8499
p.update()

In [ ]:
img = mi.render(scene, spp=32, sensor=10)
mi.util.convert_to_bitmap(img)

In [ ]:
type(cam_position.matrix)

In [ ]:
scene = mi.load_file("../scenes/intersection_taxi/intersection_taxi.xml")
depth_img = mi.render(scene, spp=32, sensor=10)

In [ ]:
# dr.shape(depth_img)
mi.Bitmap(depth_img)

In [12]:
def gen_cam_positions(z,r,size) -> np.ndarray:
    lat_r = np.sqrt(r**2 - z**2)  # find latitude circle radius
    num_points = np.arange(1,size+1)
    angles = np.array([(2 * np.pi * p / size) for p in num_points])
    vertices = np.array([np.array([np.cos(a)*lat_r, z, np.sin(a)*lat_r]) for a in angles])
    return vertices

lats = [2.1381900311, 4.1942100525,6.0890493393]
z = lats[1]
r = 11
size=32 # desired # pts on the latitude circle
vertices = gen_cam_positions(z,r,size)
print(vertices[0:5])

[[9.9736075  4.19421005 1.98387388]
 [9.39493283 4.19421005 3.8915086 ]
 [8.45521617 4.19421005 5.64959482]
 [7.19057028 4.19421005 7.19057028]
 [5.64959482 4.19421005 8.45521617]]
